# get_fdt test

## TO-DO
- sort by values or by index
- fmt_values

In [1]:
## Standard Libs
from typing import Union, Optional, Tuple, Literal, Any
import warnings
import random

# Third-Party Libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns

# # Local Libs
# from jm_datascience import jm_pandas as jm_pd
# from jm_datascience import jm_pdaccessor as jm
# from jm_utils import jm_richprt as jm_prt


## Some Series and DFs

In [2]:
df_work = pd.DataFrame({
    'nombre': ['Ana', 'Bob', '', 'Carlos', ' ', 'Diana'],
    'apellido': ['A_Ana', 'B_Bob', None, 'C_Carlos', None, 'D_Diana'],
    'edad': [25, -1, 30, 999, 28, 22],
    'ciudad': ['Madrid', 'N/A', 'Barcelona', 'Valencia', 'unknown', 'Sevilla'],
    'salario': [50000, 0, 60000, -999, 55000, 48000]
})

## Read spreedsheet for tests
try:
    spreedsheet = r"C:\Users\jm\Documents\__Dev\PortableGit\__localrepos\365DS_jm\3_statistics\2_13_Practical_Ex_Descriptive_Stats.xlsx"    # Casa
    with open(spreedsheet) as f:
        pass
except FileNotFoundError:
    spreedsheet = r"D:\git\PortableGit\__localrepos\365DS_jm\3_statistics\2_13_Practical_Ex_Descriptive_Stats.xlsx"                         # Office

df_xls = pd.read_excel(spreedsheet, skiprows=4, usecols='B:J,L:AA', index_col='ID')
df = df_xls.copy()

lst_str = random.choices([chr(i) for i in range(65, 72)], k=175)
# sr_str = jm_pd.to_series(lst_str)                         # <- jm_pd.to_serie_with_count()

In [3]:
display(df['State'].unique())
df['State'].value_counts(sort=False, ascending=True)
df['Country'].value_counts(sort=True, ascending=True, dropna=False)

array(['California', 'Virginia', 'Arizona', 'Oregon', 'Nevada',
       'Colorado', 'Utah', nan, 'Kansas', 'Wyoming'], dtype=object)

Country
Germany      1
Mexico       1
Denmark      1
UK           2
Belgium      2
Russia       4
Canada       7
USA         12
NaN         72
USA        165
Name: count, dtype: int64

In [4]:
def _fmt_value_for_pd(value, width=8, decimals=3, miles=',') -> str:
    """
    Format a value (numeric or string) into a right-aligned string of fixed width.

    Converts numeric values to formatted strings with thousands separators and
    specified decimal places. Strings are padded to the same width for consistent alignment.

    Parameters:
        value (int, float, str): The value to be formatted.
        width (int): Total width of the output string. Must be a positive integer.
        decimals (int): Number of decimal places for numeric values. Must be >= 0.
        miles (str or None): Thousands separator. Valid options: ',', '_', or None.

    Returns:
        str: The formatted string with right alignment.

    Raises:
        ValueError: If width <= 0, decimals < 0, or miles is invalid.

    Examples:
        >>> format_value(123456.789)
        '123,456.79'
        >>> format_value("text", width=10)
        '      text'
        >>> format_value(9876, miles=None)
        '    9876.00'
    """
    # Parameter Value validation <- vamos a tener que analizar este tema por si es un list , etc,,
    #   - En realidad acá tenemos que evaluar algo similar a jm_utils - fmt_values() FUTURE
    # if not isinstance(value, (int, float, np.integer, np.floating)) or pd.api.types.is_any_real_numeric_dtype(value)

    if not isinstance(width, int) or width <= 0:
        raise ValueError(f"Width must be a positive integer. Not '{width}'")
    
    if not isinstance(decimals, int) or decimals < 0:
        raise ValueError(f"Decimals must be a non-negative integer. Not '{decimals}")
    
    if miles not in [',', '_', None]:
        raise ValueError(f"Miles must be either ',', '_', or None. Not '{miles}")
    
    try:
        num = float(value)                                  # Convert to float if possible
        if num % 1 == 0:                                    # it its a total integer number
            decimals = 0
        if miles:
            return f"{num:>{width}{miles}.{decimals}f}"     # Ancho fijo, x decimales, alineado a la derecha
        else:
            return f"{num:>{width}.{decimals}f}"
        
    except (ValueError, TypeError):
        return str(value).rjust(width)                      # Alinea también strings, para mantener la grilla

In [5]:
def to_series(
    data: Union[pd.Series, np.ndarray, dict, list, set, pd.DataFrame],
    index: Optional[pd.Index] = None,
    name: Optional[str] = None
) -> pd.Series:
    """
    Converts input data into a pandas Series, optionally returning value counts.

    This function accepts various data types and converts them into a pandas Series.
    If `count=True`, it returns the frequency count of the values in the resulting Series.

    Parameters:
        data (Union[pd.Series, np.ndarray, dict, list, set, pd.DataFrame]):
            The input data to convert. Supported types include:
            - pd.Series: returned as-is or counted if `count=True`.
            - np.ndarray: flattened and converted to a Series.
            - dict: keys become the index, values are used for data.
            - list or set: converted directly to a Series.
            - pd.DataFrame:
                - 1 column: converted directly to a Series.
                - 2 columns: first column becomes the index, second becomes the values.

        count (bool or int, optional): Whether to return value counts instead of raw data.
            If True or 1, returns frequencies of each value. Default is False.

    Returns:
        pd.Series: A pandas Series representing the input data. If `count=True`, returns
            the value counts of the data.

    Raises:
        TypeError: If `data` is not one of the supported types.
        ValueError: If `count` is not a boolean or integer 0/1.
        ValueError: If DataFrame has more than 2 columns.

    Examples:
        >>> import pandas as pd
        >>> to_serie_with_count([1, 2, 2, 3])
        0    1
        1    2
        2    2
        3    3
        dtype: int64

        >>> to_serie_with_count([1, 2, 2, 3], count=True)
        2    2
        1    1
        3    1
        dtype: int64

        >>> df = pd.DataFrame({'Category': ['A', 'B', 'A'], 'Value': [10, 20, 30]})
        >>> to_serie_with_count(df)
        Category
        A    10
        B    20
        A    30
        Name: Value, dtype: int64
    """
    
    # Validate parameters - FUTURE
    
    if isinstance(data, pd.Series):                 # If data is already a series no conversion needed
        sr = data                                  
    elif isinstance(data, np.ndarray):              # If data is a NumPy array   
        sr = pd.sr(data.flatten())
    elif isinstance(data, (dict, list)):
        sr = pd.sr(data)
    elif isinstance(data, (set)):
        sr = pd.sr(tuple(data))
    elif isinstance(data, pd.DataFrame):
        if data.shape[1] == 1:                      # Also len(data.columns == 1)
            sr = data.iloc[:, 0]
        elif data.shape[1] == 2:                    # Index: first col, Data: 2nd Col
            sr = data.set_index(data.columns[0])[data.columns[1]]
        else:
            raise ValueError("DataFrame must have 1 oer 2 columns. Categories and values for 2 columns cases.")
    else:
        raise TypeError(f"Unsupported data type: {type(data)}. "
                    "Supported types: pd.sr, np.ndarray, pd.DataFrame, dict, list, set, and pd.DataFrame")

    if name:
        sr.name = name

    if index:
        sr.index = index

    return sr

In [6]:
def get_fdt(
        data: Union[pd.Series, np.ndarray, dict, list, pd.DataFrame],
        value_counts: Optional[bool] = False,
        dropna: Optional[bool] = True,
        na_position: Optional[str] = 'last',
        pcts: Optional[bool] = True,
        plain_relatives: Optional[bool] = True,
        fmt_values: Optional[bool] = False,
        sort: Optional[str] = 'desc',
        na_aside: Optional[bool] = True
) -> pd.DataFrame:
    '''
    Generata a Frequency Distribution Table (fdt)

    data: puede ser ya con el value_counts() hecho o no
    sort: 'desc', 'asc', 'ix_asc', 'ix_desc', para como queremos que sea vea el orden por valores o por indice
    na_position: 'first', 'last', 'value'
    '''
    columns = [
        'Frequency',
        'Cumulative Frequency',
        'Relative Frequency',
        'Cumulative Relative Freq.',
        'Relative Freq. [%]',
        'Cumulative Freq. [%]'
    ]
    # def _calculate_fdt_relatives(series):     # Revisar, no me gusta el flujo actual
    
    sr = to_series(data)
    
    if dropna:
        sr = sr.dropna()

    if value_counts:
        sr = sr.value_counts(dropna=dropna, sort=False)

    match sort:
        case 'asc':
            sr = sr.sort_values()
        case 'desc':
            sr = sr.sort_values(ascending=False)
        case 'ix_asc':
            sr = sr.sort_index()
        case 'ix_desc':
            sr = sr.sort_index(ascending=False)
        case None:
            pass
        case _:
            raise ValueError(f"Valid values for sort: 'asc', 'desc', 'ix_asc', 'ix_desc', or None. Got '{sort}'")

    try:                            # To manage when there aren't NaNs
        nan_value = sr[np.nan]
        sr_without_nan = sr.drop(np.nan)
    except:
        pass
    else:                           # if NaNs: 1. na_position, 2 na_count
        match na_position:          # 1. locate the NaNs values
            case 'first':
                sr = pd.concat([pd.Series({np.nan: nan_value}), sr_without_nan])
            case 'last':
                sr = pd.concat([sr_without_nan, pd.Series({np.nan: nan_value})])
            case 'value' | None:
                pass
            case _:
                raise ValueError(f"Valid values for na_position: 'first', 'last', 'value' or None. Got '{na_position}'")
        
        if na_aside:                # 2. define if NaNs count for relative and cumulative values.
            sr = sr_without_nan     # series without nulls on which the relative values will be calculated
            # Column that will then be concatenated to the end of the DF if the na_aside option is true
            nan_row_df = pd.DataFrame(data = [nan_value], columns=[columns[0]], index=['Nulls'])      # Only 'Frequency' column, others empty

    fdt = pd.DataFrame(sr)
    fdt.columns = [columns[0]]
    fdt[columns[1]] = fdt['Frequency'].cumsum()
    fdt[columns[2]] = fdt['Frequency'] / fdt['Frequency'].sum()
    fdt[columns[3]] = fdt['Relative Frequency'].cumsum()
    fdt[columns[4]] = fdt['Relative Frequency'] * 100
    fdt[columns[5]] = fdt['Cumulative Relative Freq.'] * 100

    if na_aside and not dropna:      # We add nan_columns at the end
        fdt = pd.concat([fdt, nan_row_df])

    if not pcts:                    # Don't return percentage columns
        fdt = fdt[columns[0:4]]
    
    if not plain_relatives:         # Don't return relative and plain cumulative
        fdt = fdt[[columns[0], columns[4], columns[5]]]

    if fmt_values:
        fdt = fdt.map(_fmt_value_for_pd)
        
    return fdt

In [15]:
fdt_s1 = get_fdt(df['Country'], value_counts=True, sort='asc', dropna=False, na_position='value', fmt_values=True, na_aside=False)
fdt_s1



,Frequency,Cumulative Frequency,Relative Frequency,Cumulative Relative Freq.,Relative Freq. [%],Cumulative Freq. [%]
Country,,,,,,
Germany,1,1,0.004,0.004,0.375,0.375
Mexico,1,2,0.004,0.007,0.375,0.749
Denmark,1,3,0.004,0.011,0.375,1.124
UK,2,5,0.007,0.019,0.749,1.873
Belgium,2,7,0.007,0.026,0.749,2.622
Russia,4,11,0.015,0.041,1.498,4.120
Canada,7,18,0.026,0.067,2.622,6.742
USA,12,30,0.045,0.112,4.494,11.236
NaN,72,102,0.270,0.382,26.966,38.202


In [8]:
fdt_2 = get_fdt(df['State'], value_counts=True)    
cumulative_pcts = fdt_2['Cumulative Freq. [%]']
top_3_pct = cumulative_pcts.iloc[min(2, len(cumulative_pcts)-1)]

labels = [f"{fdt_2.iloc[ix, 0]} ({fdt_2.iloc[ix, -2]:.1f} %)" for ix in range(fdt_2.shape[0])]
print(labels)

for iloc_ix in range(len(cumulative_pcts)):
    print(f"cumulative_pcts.iloc[{iloc_ix}] = {cumulative_pcts.iloc[iloc_ix]}")

display(len(cumulative_pcts))
display(top_3_pct)
fdt_2

['119 (65.7 %)', '17 (9.4 %)', '11 (6.1 %)', '11 (6.1 %)', '11 (6.1 %)', '6 (3.3 %)', '4 (2.2 %)', '1 (0.6 %)', '1 (0.6 %)']
cumulative_pcts.iloc[0] = 65.74585635359117
cumulative_pcts.iloc[1] = 75.13812154696133
cumulative_pcts.iloc[2] = 81.21546961325966
cumulative_pcts.iloc[3] = 87.292817679558
cumulative_pcts.iloc[4] = 93.37016574585634
cumulative_pcts.iloc[5] = 96.68508287292816
cumulative_pcts.iloc[6] = 98.89502762430938
cumulative_pcts.iloc[7] = 99.44751381215468
cumulative_pcts.iloc[8] = 99.99999999999997


9

np.float64(81.21546961325966)

,Frequency,Cumulative Frequency,Relative Frequency,Cumulative Relative Freq.,Relative Freq. [%],Cumulative Freq. [%]
State,,,,,,
California,119,119,0.657459,0.657459,65.745856,65.745856
Nevada,17,136,0.093923,0.751381,9.392265,75.138122
Arizona,11,147,0.060773,0.812155,6.077348,81.215470
Oregon,11,158,0.060773,0.872928,6.077348,87.292818
Colorado,11,169,0.060773,0.933702,6.077348,93.370166
Utah,6,175,0.033149,0.966851,3.314917,96.685083
Virginia,4,179,0.022099,0.988950,2.209945,98.895028
Kansas,1,180,0.005525,0.994475,0.552486,99.447514
Wyoming,1,181,0.005525,1.000000,0.552486,100.000000
